In [1]:
import os
import numpy as np
import random
import json
import yaml
import torch

In [2]:
import sys
sys.path.append("../")
from Data.CarlaSMNet import *
from utils import *
sys.path.append("../../EndToEnd/")
from Segmentation.spvcnn import *

In [6]:
# MODEL_CONFIG = "SMNet"
MODEL_CONFIG = "SMNetKITTI_Transfer"
# DATA_CONFIG = "Carla"
DATA_CONFIG = "KITTI"

In [7]:
model_params_file = os.path.join(os.getcwd(), "../Configs", MODEL_CONFIG + ".yaml")
with open(model_params_file, "r") as stream:
    try:
        model_params = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

data_params_file = os.path.join(os.getcwd(), "../Configs", DATA_CONFIG + ".yaml")
with open(data_params_file, "r") as stream:
    try:
        data_params = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

T = model_params["T"]
ds = model_params["ego_downsample"]
mem_feat_dim = model_params["mem_feat_dim"]
ego_feat_dim = model_params["ego_feat_dim"]
train_dir = data_params["train_dir"]
val_dir = data_params["val_dir"]
test_dir = data_params["test_dir"]
num_workers = data_params["num_workers"]
n_obj_classes = 19 # data_params["num_classes"]
remap = data_params["remap"]
seed = model_params["seed"]

coor_ranges = data_params['min_bound'] + data_params['max_bound']
voxel_sizes = [abs(coor_ranges[3] - coor_ranges[0]) / data_params["x_dim"],
              abs(coor_ranges[4] - coor_ranges[1]) / data_params["y_dim"],
              abs(coor_ranges[5] - coor_ranges[2]) / data_params["z_dim"]]

In [13]:
seg_net = SPVCNN(
    num_classes=n_obj_classes,
    cr=model_params["cr"],
    pres=model_params["pres"],
    vres=model_params["vres"]).to(device)

# seg_net.load_state_dict(torch.load(model_params["weights_path"])['model'])
seg_net.load_state_dict(torch.load(model_params["weights_path"]))
seg_net = remove_classifier(seg_net)
seg_net.eval()

SPVCNN(
  (stem): Sequential(
    (0): Conv3d(4, 16, kernel_size=(3, 3, 3), bias=False)
    (1): BatchNorm(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv3d(16, 16, kernel_size=(3, 3, 3), bias=False)
    (4): BatchNorm(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (stage1): Sequential(
    (0): BasicConvolutionBlock(
      (net): Sequential(
        (0): Conv3d(16, 16, kernel_size=(2, 2, 2), stride=(2, 2, 2), bias=False)
        (1): BatchNorm(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (1): ResidualBlock(
      (net): Sequential(
        (0): Conv3d(16, 16, kernel_size=(3, 3, 3), bias=False)
        (1): BatchNorm(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv3d(16, 16, kernel_size=(3, 3, 3), bias=False)
        (4): BatchNorm

In [8]:
# CARLA
split_dir = "/home/tigeriv/Data/Carla/Cartesian/Test"
split_dir = "/home/tigeriv/Data/KITTI/dataset/sequences"
for scene in os.listdir(split_dir):
    print(scene)
    scene_dir = os.path.join(split_dir, scene, "cartesian")
    velo_dir = os.path.join(scene_dir, "velodyne")
    save_dir = os.path.join(scene_dir, "SMNetInput")
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    for velo_file in sorted(os.listdir(velo_dir)):
        index = velo_file.split(".")[0]
        lidar = np.fromfile(os.path.join(velo_dir, velo_file), dtype=np.float32).reshape(-1, 4)
        seg_input, inv = generate_seg_in(lidar, model_params["vres"])
        point_features = seg_net(seg_input.to(device))[inv].detach().cpu().numpy().astype(np.float16) # save memory
        save_file = os.path.join(save_dir, index + ".bin")
        point_features.tofile(save_file) # Would be 20 MB each!!!

Town10_Light
Town10_Heavy
Town10_Medium


In [14]:
# KITTI
split_dir = "/home/tigeriv/Data/KITTI/dataset/sequences"
for scene in os.listdir(split_dir):
    print(scene)
    scene_dir = os.path.join(split_dir, scene)
    velo_dir = os.path.join(scene_dir, "velodyne")
    save_dir = os.path.join(scene_dir, "SMNetInput_Transfer")
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    for velo_file in sorted(os.listdir(velo_dir)):
        index = velo_file.split(".")[0]
        lidar = np.fromfile(os.path.join(velo_dir, velo_file), dtype=np.float32).reshape(-1, 4)
        seg_input, inv = generate_seg_in(lidar, model_params["vres"])
        point_features = seg_net(seg_input.to(device))[inv].detach().cpu().numpy().astype(np.float16) # save memory
        save_file = os.path.join(save_dir, index + ".bin")
        point_features.tofile(save_file) # Would be 20 MB each!!!

08
